# Classify wine data.

It is classified as 0,1 considering alcohol, sugar content, and pH characteristics.

In [ ]:
import pandas as pd
import numpy as np


from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.inspection import permutation_importance # 중요특성을 변환기

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import os

# wine_csv_data uploaded...
for dirname, _, filenames in os.walk('../input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


wine = pd.read_csv('../input/wine-scv-data/wine_csv_data.csv')

print(wine.head())
print(wine.info())
print(wine.describe())

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

print(data[:5])
print(target[:5])

In [ ]:
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# LogisticRegresssion

In [ ]:
ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

lr = LogisticRegression()
lr.fit(train_scaled, train_target)
trainscore = lr.score(train_scaled, train_target)
testscore = lr.score(test_scaled, test_target)

plt.bar((0,1),(trainscore,testscore), color=('red','orange'), linewidth=3, width=0.3)
#plt.xticks((0,1), (trainscore,testscore))
plt.title('logisticRegression')
red_patch = mpatches.Patch(color='red', label='trainscore')
orange_patch = mpatches.Patch(color='orange', label='testscore')
plt.legend(handles=[red_patch,orange_patch])

plt.show()

# DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(random_state=42,max_depth=3)
dt.fit(train_input, train_target)

scores = cross_validate(dt, train_input, train_target)
plt.plot(scores['fit_time'])
plt.title('fit_time')
plt.show()
plt.plot(scores['score_time'],label='score_time')
plt.title('score_time')
plt.show()
plt.plot(scores['test_score'],label='test_score')
plt.title('test_score')
plt.show()

plt.figure(figsize=(10,7),dpi=300)
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

# GridSearchCV

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

plt.plot(gs.cv_results_['mean_test_score'])
plt.show()

# RandomizedSearchCV

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

plt.plot(gs.cv_results_['mean_test_score'])
plt.show()

# RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

# RandomForestClassifier oob_score=True 

위 속성은 훈련 데이터로 사용하지 않은 데이터로 샘플데이터로 점수를 확인

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

# ExtraTreesClassifier

In [ ]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

In [ ]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) # tree 갯수를 500개로 지정
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(train_input, train_target)
print(gb.feature_importances_)

# HistGradientBoostingClassifier

In [ ]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# permutation_importance

In [ ]:
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)


hgb.score(test_input, test_target)

# XGBClassifier

In [ ]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# LGBMClassifier

In [ ]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))